#### Imports

In [2]:
import pandas as pd

#### Read linkedin csv

In [3]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample()

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
0,3313895534,Signal Processing AI/Machine Learning Engineer,CyberCoders,"· Raleigh, NC",100.000 $/año - 150.000 $/año,Presencial,Jornada completa,Intermedio,De 201 a 500 empleados · Dotación y selección ...
